## **Mapping Fire Impacts - Raster-based**

The Creek Fire in California ocurred in 2020. We used pre-fire (2019) data and post-fire (2021) to identify burn extent and severity. To do this, we used Normalized burn ratio (NBR) which includes Near Infrared (NIR) and Shortwave Infrared (SWIR).

                 NBR = (NIR - SWIR) / NIR + SWIR

                  from here, we can calcuate dNBR:

                 dNBR = NBRprefire - NBRpostfire



#### Reflection exercises for each group:

1. At the top of the Colab notebook that you share with everyone, please include your responses to the following questions:
  - Choose at least 1 dataset to explore in more detail. **We are looking at NEON datasets for the SOAP site showing RGB and SDR data.**
  - What is the projection for this dataset? **EPSG:32611** **WGS 84 / UTM zone 11N**
 - Where can you find more information on how the data were collected and how to interpret the metadata? **SOAP_2019_sdr.getInfo(). Date of flight, time of day, location, weather**
  - Think about what data type each variable is. **Coordinates, dates, etc**
  - Is it vector or raster data? What properties exist for each dataset? **Raster data (because it is an image[pixels])- SOAP/MTBS, Vector data - MTBS burn boundary.**
  - What resolution are your data? **Spatial data are 1 km mosaics; location is California; temporal resolution is annual**

2. At the top of the Colab notebook, write a short summary detailing the processing steps in the notebook and your results.
  - Although these topics may be far removed from your own interests, how could these steps and analyses help in your own work?


3. OPTIONAL - Expand your script by adding additional processing, analysis, or other data. **We added a raster file from MTBS showing the NBS to compare to our results to confirm accuracy.**

As you're working through your exercise, **add code chunks to further document your scripts. Add additional comments to the code itself to clarify complicated processes.**




---



#### Acknowledgement

The code and comments for this project were adpated from NEON in [this tutorial](https://www.neonscience.org/resources/learning-hub/tutorials/intro-aop-data-google-earth-engine-tutorial-series#toggle-36). Check it out (or any of their other awesome tutorials!) to learn more!

### Objectives

In this first section, we'll explore the Soaproot Saddle NEON site in California before and after a 2020 burn. We'll map the reflectance data, RGB data, and difference in normalized burn ratio (dNBR).

<div id="ds-objectives" markdown="1">

### Background

The **Creek Fire** was a large wildfire that started in September 2020 in the Sierra National Forest, California and became one of the largest fires of the [2020 California wildfire season](https://en.wikipedia.org/wiki/2020_California_wildfires). This fire had burned into NEON's [Soaproot Saddle (SOAP)](https://www.neonscience.org/field-sites/soap) site by mid-September, causing a [high intensity burn over much of the site](https://www.neonscience.org/impact/observatory-blog/domain-digest-no-4-fire-field-sites-and-recovery) - standing trees became charcoal spires, shrubs and their root systems burned completely, the thick litter layer was incinerated, and the soil severely burned.

The NEON Airborne Observation Platform (AOP) conducted aerial surveys over SOAP in 2019 and 2021, a year before and after the Creek Fire.

In our introduction to AOP data, we learned how to calculate NDVI. Another common vegetation index is NBR, the normalized burn ratio. NBR is commonly used to asses burn severity. NBR is calculated as: <br>
> NBR = (NIR – SWIR) / (NIR + SWIR)

<img src='https://drive.google.com/uc?export=view&id=1zUP9oDdeQdiyQGL-2Vh1aqz8QCy03XPR' width=400> Image credit: [NASA ARSET Training Materials](https://appliedsciences.nasa.gov/sites/default/files/ChangeDetection_S1.pdf)


Then, dNBR is the difference in NBR, which is calcualted as <br>
> dNBR = prefire NBR - postfire NBR

The USGS proposes the following categories based on dNBR:<br>
<img src='https://drive.google.com/uc?export=view&id=1firJsT-ON56_LYII9MErCw1UC6V8Y3QD' width=600>Image credit: [Earth Lab](https://www.earthdatascience.org/courses/earth-analytics/multispectral-remote-sensing-modis/normalized-burn-index-dNBR/)

  - [Surface Directional Reflectance](https://data.neonscience.org/data-products/DP3.30006.001)
  - [High resolution camera imagery, red-green-blue (RGB)](https://data.neonscience.org/data-products/DP3.30010.001)
  - [Canopy height model](https://data.neonscience.org/data-products/DP3.30015.001)
  - [Digital elevation model](https://data.neonscience.org/data-products/DP3.30024.001)


In [ ]:
import ee
import geemap
import geemap.colormaps as cm
ee.Authenticate()
ee.Initialize(project='820081621265')
geemap.ee_initialize(project='')

In [ ]:
# Import the AOP surface directional reflectance (SDR)
aopSDR = ee.ImageCollection('projects/neon-prod-earthengine/assets/DP3-30006-001') # Raw data with 442 bands.
# Import the AOP red-green-blue imagery (RGB)
aopRGB = ee.ImageCollection('projects/neon-prod-earthengine/assets/DP3-30010-001')
# Import the AOP canopy height model data (CHM)
aopCHM = ee.ImageCollection('projects/neon-prod-earthengine/assets/DP3-30015-001')
# Import the AOP digital elevation model data (DEM)
aopDEM = ee.ImageCollection('projects/neon-prod-earthengine/assets/DP3-30024-001')

# Set up visualization params
RGB_bands = ['B053', 'B035', 'B019'] # These are the band names for the red, green, blue bands in the SDR data
rgbVis = {'min': 0, 'max': 255, 'gamma': 0.8} # This sets a nice range of values for mapping the RGB data
sdrVis = {'min': 0, 'max': 1260, 'gamma': 0.8} # This sets a nice range of values for mapping the RGB data

In [ ]:
### Code and comments adpated from NEON at https://www.neonscience.org/resources/learning-hub/tutorials/intro-aop-data-google-earth-engine-tutorial-series#toggle-36
'''------------------------- Normalized Difference Burn Ratio ----------------------------
The normalized burn ratio (NBR) is a normalized difference index using the shortwave-infrared (SWIR) and near-infrared (NIR) portions of the electromagnetic spectrum.
dNBR can be used as a metric to map fire extent and burn severity when calculating the difference between pre and post fire conditions.
'''

# Import the 2019 and 2021 SOAP data
SOAP_2019_sdr = aopSDR \
  .filterDate('2019-01-01', '2019-12-31') \
  .filterMetadata('NEON_SITE', 'equals', 'SOAP') \
  .first()

SOAP_2021_sdr = aopSDR \
  .filterDate('2021-01-01', '2021-12-31') \
  .filterMetadata('NEON_SITE', 'equals', 'SOAP') \
  .first()

SOAP_2019_rgb = aopRGB \
  .filterDate('2019-01-01', '2019-12-31') \
  .filterMetadata('NEON_SITE', 'equals', 'SOAP') \
  .first()


SOAP_2021_rgb = aopRGB \
  .filterDate('2021-01-01', '2021-12-31') \
  .filterMetadata('NEON_SITE', 'equals', 'SOAP') \
  .first()


## Create an ROI that talks only the pixels covered by both years + adds -30m buffer
full_mask = ee.Image.constant(1).clip(SOAP_2019_sdr.geometry()) \
                .updateMask(SOAP_2019_sdr.select(['B001']).mask()) \
                .updateMask(SOAP_2021_sdr.select(['B001']).mask()) \
                .reduceToVectors(maxPixels=1e13, scale=100)
ROI = full_mask.geometry().buffer(-30)

# calculate NBR
# B097: B365:
pre_nbr = SOAP_2019_sdr.normalizedDifference(['B097', 'B365'])
post_nbr = SOAP_2021_sdr.normalizedDifference(['B097', 'B365'])

# calculate dNBR
dNBR = pre_nbr \
        .subtract(post_nbr)

# Mapping
vis_params = {'min':-1, 'max':1, 'palette':cm.get_palette('RdYlGn_r', n_class=20)}
m = geemap.Map()


# # add RGB base
m.addLayer(SOAP_2019_rgb, rgbVis, 'SOAP 2019 RGB')
m.addLayer(SOAP_2021_rgb, rgbVis, 'SOAP 2021 RGB')

# # add NBR layers
m.addLayer(pre_nbr, vis_params, '2019 Normalized Burn Ratio')
m.addLayer(post_nbr, vis_params, '2021 Normalized Burn Ratio')

# add dNBR layer
m.addLayer(dNBR, vis_params, 'dNBR')

m.centerObject(ROI)
m

In [ ]:
SOAP_2019_sdr.projection() # How to find the projection.

In [ ]:
SOAP_2019_sdr.getInfo() # This is how to find the metadata.

display('All metadata:', SOAP_2019_sdr) # A different way to display the metadata of a single image.

# Bringing in vector data (fire exploration)
Now, let's compare our map of dNBR with existing fire data. The MTBS (Monitoring Trends in Burn Severity) data product is already publicly available on GEE.

In the next code chunk, we'll inspect 1 feature (representing a fire polygon) from MTBS. We extract the feature of interested by filtering the MTBS FeatureCollection to our area of interest and the year 2020.

In [ ]:
MTBS = ee.FeatureCollection("USFS/GTAC/MTBS/burned_area_boundaries/v1") \
              .filterBounds(SOAP_2021_sdr.geometry()) \
              .map(lambda feat: feat.set('system:time_start', feat.get('Ig_Date'))) \
              .filterDate('2020-01-01', '2021-01-01')
MTBS

Now, let's overlay the MTBS data product on our dNBR map.

In [ ]:
mtbs_creek = (ee.ImageCollection("USFS/GTAC/MTBS/annual_burn_severity_mosaics/v1") #From gee, Monitoring Trends in Burn Severity (MTBS) Burn Severit
                    .filter(ee.Filter.eq('system:index','mtbs_CONUS_2020'))
)

sev = ee.Image(mtbs_creek.first()).clip(SOAP_2019_rgb.geometry()) #You can take the mtbs_creek image and do a clip function so that it only shows within the SOAP site.

In [ ]:
ee.ImageCollection("USFS/GTAC/MTBS/annual_burn_severity_mosaics/v1")

In [ ]:
# Mapping
vis_params = {'min':-1, 'max':1, 'palette':cm.get_palette('RdYlGn_r', n_class=20)}
m = geemap.Map()


# # add RGB base
m.addLayer(SOAP_2019_rgb, rgbVis, 'SOAP 2019 RGB')
m.addLayer(SOAP_2021_rgb, rgbVis, 'SOAP 2021 RGB')

# add dNBR layer
m.addLayer(dNBR, vis_params, 'dNBR')

# add MTBS layer
m.addLayer(MTBS)
m.addLayer(sev)

m.centerObject(ROI)
m

Now that we've seen the boundary of the 2020 Creek Fire, let's use the canopy 2019 and 2021 height model (CHM) and the Creek Fire Boundary to see the change in canopy height post-fire at the SOAP site.

First, we'll show the distribution of changes in canopy height post-fire for the entire burn region.

Then, we'll use dNBR thresholds to estimate "high severity" vs "low severity" burn regions within the burned area, and plot the distribution of changes in canopy height by estimated severity. <br>
*Note: If you're interested in fire severity classifications, MTBS also has a burn severity raster product available on GEE!*

In [ ]:
# Read in the CHM for 2019 and 2021
SOAP_2019_chm = aopCHM \
  .filterDate('2019-01-01', '2019-12-31') \
  .filterMetadata('NEON_SITE', 'equals', 'SOAP') \
  .first()


SOAP_2021_chm = aopCHM \
  .filterDate('2021-01-01', '2021-12-31') \
  .filterMetadata('NEON_SITE', 'equals', 'SOAP') \
  .first()

# Convert rasterize the MTBS polygon
creek_img = MTBS.reduceToImage(properties=['BurnBndAc'], reducer=ee.Reducer.anyNonZero()).clip(ROI).rename('burned')


# Calculate the pixel-by-pixel difference in canopy height between 2019 and 2021 for only the area in the Creek fire burn region
chm_diff_burned = SOAP_2021_chm.subtract(SOAP_2019_chm) \
                    .updateMask(creek_img.eq(1)) \
                    .sample(region=ROI, scale=30)

# Calculate the pixel-by-pixel difference in canopy height between 2019 and 2021 for only the area outside the Creek fire burn region
chm_diff_unburned = SOAP_2021_chm.subtract(SOAP_2019_chm) \
                    .updateMask(creek_img.eq(0)) \
                    .sample(region=ROI, scale=30)

# Plot difference in CHM
import geemap.chart as chart

options = {
    "xlabel": "Change in canopy height (m)",
    "ylabel": "Pixel count",
    "colors": ["#1d6b99"],
}
chart.feature_histogram(chm_diff_burned, 'CHM', title='Change in canopy height post-fire in the Creek Fire boundary', **options)

In [ ]:
chart.feature_histogram(chm_diff_unburned, 'CHM', title='Change in canopy height post-fire outside the Creek Fire boundary', **options)

<details>
<summary><h2>MAP EXPORT TROUBLESHOOTING LOG USING CHAT GPT</h2></summary>

In short, I took the code for the map object we have just created in colab and pasted into chatgpt. Then I asked how can I export the map object into google drive. It provided a code to do this based on the information provided, but also a generic code with placeholders for the information it does not have. After some troubleshooting I was able to achieve the export. The colors came out incorrect, so it requires more troubleshooting.

User
# Import the 2019 and 2021 SOAP data
SOAP_2019_sdr = aopSDR \
  .filterDate('2019-01-01', '2019-12-31') \
  .filterMetadata('NEON_SITE', 'equals', 'SOAP') \
  .first()

SOAP_2021_sdr = aopSDR \
  .filterDate('2021-01-01', '2021-12-31') \
  .filterMetadata('NEON_SITE', 'equals', 'SOAP') \
  .first()

SOAP_2019_rgb = aopRGB \
  .filterDate('2019-01-01', '2019-12-31') \
  .filterMetadata('NEON_SITE', 'equals', 'SOAP') \
  .first()


SOAP_2021_rgb = aopRGB \
  .filterDate('2021-01-01', '2021-12-31') \
  .filterMetadata('NEON_SITE', 'equals', 'SOAP') \
  .first()


## Create an ROI that talks only the pixels covered by both years + adds -30m buffer
full_mask = ee.Image.constant(1).clip(SOAP_2019_sdr.geometry()) \
                .updateMask(SOAP_2019_sdr.select(['B001']).mask()) \
                .updateMask(SOAP_2021_sdr.select(['B001']).mask()) \
                .reduceToVectors(maxPixels=1e13, scale=100)
ROI = full_mask.geometry().buffer(-30)

# calculate NBR
# B097: B365:
pre_nbr = SOAP_2019_sdr.normalizedDifference(['B097', 'B365'])
post_nbr = SOAP_2021_sdr.normalizedDifference(['B097', 'B365'])

# calculate dNBR
dNBR = pre_nbr \
        .subtract(post_nbr)

# Mapping
vis_params = {'min':-1, 'max':1, 'palette':cm.get_palette('RdYlGn_r', n_class=20)}
m = geemap.Map()


# # add RGB base
m.addLayer(SOAP_2019_rgb, rgbVis, 'SOAP 2019 RGB')
m.addLayer(SOAP_2021_rgb, rgbVis, 'SOAP 2021 RGB')

# # add NBR layers
m.addLayer(pre_nbr, vis_params, '2019 Normalized Burn Ratio')
m.addLayer(post_nbr, vis_params, '2021 Normalized Burn Ratio')

# add dNBR layer
m.addLayer(dNBR, vis_params, 'dNBR')

m.centerObject(ROI)
m How can I export this ap to google drive from colab and using earth engine functions
ChatGPT
To export the dNBR image or other layers to Google Drive from Google Colab using Earth Engine functions, you'll need to use the ee.batch.Export.image.toDrive function. Here are the steps to do so:

Authenticate and Initialize Earth Engine: Ensure you have authenticated and initialized the Earth Engine library in your Colab notebook.
Set up the Export Parameters: Define the parameters for exporting the image to Google Drive.
Start the Export Task: Use ee.batch.Export.image.toDrive to start the export task.
Here's how you can do it step-by-step:

Step 1: Authenticate and Initialize Earth Engine in Google Colab
First, ensure you have authenticated and initialized Earth Engine in your Colab notebook.

python
Copy code
import ee
ee.Authenticate()
ee.Initialize()
Step 2: Define the Export Parameters
Set up the parameters for exporting the dNBR image. You can specify the region of interest, scale, and other export options.

Step 3: Start the Export Task
Use ee.batch.Export.image.toDrive to export the image to Google Drive.

Here is the complete code:

python
Copy code
# Import the Earth Engine library
import ee

# Authenticate and initialize the Earth Engine library
ee.Authenticate()
ee.Initialize()

# Define the SOAP data for 2019 and 2021
aopSDR = ee.ImageCollection('path/to/aopSDR')  # Replace with actual path
aopRGB = ee.ImageCollection('path/to/aopRGB')  # Replace with actual path

SOAP_2019_sdr = aopSDR.filterDate('2019-01-01', '2019-12-31').filterMetadata('NEON_SITE', 'equals', 'SOAP').first()
SOAP_2021_sdr = aopSDR.filterDate('2021-01-01', '2021-12-31').filterMetadata('NEON_SITE', 'equals', 'SOAP').first()

SOAP_2019_rgb = aopRGB.filterDate('2019-01-01', '2019-12-31').filterMetadata('NEON_SITE', 'equals', 'SOAP').first()
SOAP_2021_rgb = aopRGB.filterDate('2021-01-01', '2021-12-31').filterMetadata('NEON_SITE', 'equals', 'SOAP').first()

# Create an ROI that includes only the pixels covered by both years + adds -30m buffer
full_mask = ee.Image.constant(1).clip(SOAP_2019_sdr.geometry()) \
                .updateMask(SOAP_2019_sdr.select(['B001']).mask()) \
                .updateMask(SOAP_2021_sdr.select(['B001']).mask()) \
                .reduceToVectors(maxPixels=1e13, scale=100)
ROI = full_mask.geometry().buffer(-30)

# Calculate NBR
pre_nbr = SOAP_2019_sdr.normalizedDifference(['B097', 'B365'])
post_nbr = SOAP_2021_sdr.normalizedDifference(['B097', 'B365'])

# Calculate dNBR
dNBR = pre_nbr.subtract(post_nbr)

# Define export parameters
export_params = {
    'image': dNBR,
    'description': 'dNBR_Export',
    'scale': 30,
    'region': ROI,
    'fileFormat': 'GeoTIFF',
    'maxPixels': 1e13,
    'folder': 'GEE_Exports'  # Folder in your Google Drive
}

# Start the export task
export_task = ee.batch.Export.image.toDrive(**export_params)
export_task.start()

# Check the status of the task
import time
while export_task.active():
    print('Polling for task (id: {}).'.format(export_task.id))
    time.sleep(30)
Explanation:
Import and Authenticate:

import ee
ee.Authenticate() and ee.Initialize() are used to authenticate and initialize the Earth Engine API.
Define SOAP Data:

Load the aopSDR and aopRGB image collections.
Filter for the specific dates and site metadata to get the images for 2019 and 2021.
Create ROI:

Create a region of interest that covers only the pixels present in both years, and add a -30m buffer.
Calculate NBR and dNBR:

Compute the Normalized Burn Ratio (NBR) for both years.
Compute the difference in NBR (dNBR).
Export Parameters:

Define the parameters for the export, including the image to export, description, scale, region, file format, and Google Drive folder.
Start Export Task:

Start the export task using ee.batch.Export.image.toDrive.
Polling Task Status:

Optionally, you can poll the status of the export task to know when it completes.
By following these steps, you can export your Earth Engine results directly to Google Drive from Google Colab. Ensure that you replace placeholders like 'path/to/aopSDR' and 'GEE_Exports' with actual paths and folder names as per your data and structure.

User
---------------------------------------------------------------------------
HttpError                                 Traceback (most recent call last)
/usr/local/lib/python3.10/dist-packages/ee/data.py in _execute_cloud_call(call, num_retries)
    401   try:
--> 402     return call.execute(num_retries=num_retries)
    403   except googleapiclient.errors.HttpError as e:

6 frames
HttpError: <HttpError 400 when requesting https://earthengine.googleapis.com/v1/projects/ee-seeds/image:export?alt=json returned "ImageCollection.load: ImageCollection asset 'path/to/aopSDR' not found (does not exist or caller does not have access).". Details: "ImageCollection.load: ImageCollection asset 'path/to/aopSDR' not found (does not exist or caller does not have access).">

During handling of the above exception, another exception occurred:

EEException                               Traceback (most recent call last)
/usr/local/lib/python3.10/dist-packages/ee/data.py in _execute_cloud_call(call, num_retries)
    402     return call.execute(num_retries=num_retries)
    403   except googleapiclient.errors.HttpError as e:
--> 404     raise _translate_cloud_exception(e)  # pylint: disable=raise-missing-from
    405
    406

EEException: ImageCollection.load: ImageCollection asset 'path/to/aopSDR' not found (does not exist or caller does not have access).
/content/sample_data.
ChatGPT
The error you're encountering indicates that the ImageCollection asset 'path/to/aopSDR' does not exist or you don't have access to it. You need to specify the correct path to the ImageCollection you want to use. For example, if you're using a specific NEON dataset, you need to use the correct Earth Engine asset ID.

Here’s an updated example assuming you are using some specific public NEON datasets available in Earth Engine. Ensure you replace the placeholders with actual dataset IDs you have access to.

Example with Correct Asset IDs:
Authenticate and Initialize Earth Engine:
python
Copy code
import ee
ee.Authenticate()
ee.Initialize()
Define the Image Collections and the Script:
python
Copy code
# Define the image collections (replace with the actual paths if different)
aopSDR = ee.ImageCollection('NEON/D02/OSBS/2016_SDR')
aopRGB = ee.ImageCollection('NEON/D02/OSBS/2016_RGB')

# Import the 2019 and 2021 SOAP data
SOAP_2019_sdr = aopSDR.filterDate('2019-01-01', '2019-12-31').filterMetadata('NEON_SITE', 'equals', 'SOAP').first()
SOAP_2021_sdr = aopSDR.filterDate('2021-01-01', '2021-12-31').filterMetadata('NEON_SITE', 'equals', 'SOAP').first()

SOAP_2019_rgb = aopRGB.filterDate('2019-01-01', '2019-12-31').filterMetadata('NEON_SITE', 'equals', 'SOAP').first()
SOAP_2021_rgb = aopRGB.filterDate('2021-01-01', '2021-12-31').filterMetadata('NEON_SITE', 'equals', 'SOAP').first()

# Create an ROI that includes only the pixels covered by both years + adds -30m buffer
full_mask = ee.Image.constant(1).clip(SOAP_2019_sdr.geometry()) \
                .updateMask(SOAP_2019_sdr.select(['B001']).mask()) \
                .updateMask(SOAP_2021_sdr.select(['B001']).mask()) \
                .reduceToVectors(maxPixels=1e13, scale=100)
ROI = full_mask.geometry().buffer(-30)

# Calculate NBR
pre_nbr = SOAP_2019_sdr.normalizedDifference(['B097', 'B365'])
post_nbr = SOAP_2021_sdr.normalizedDifference(['B097', 'B365'])

# Calculate dNBR
dNBR = pre_nbr.subtract(post_nbr)

# Define export parameters
export_params = {
    'image': dNBR,
    'description': 'dNBR_Export',
    'scale': 30,
    'region': ROI,
    'fileFormat': 'GeoTIFF',
    'maxPixels': 1e13,
    'folder': 'GEE_Exports'  # Folder in your Google Drive
}

# Start the export task
export_task = ee.batch.Export.image.toDrive(**export_params)
export_task.start()

# Check the status of the task
import time
while export_task.active():
    print('Polling for task (id: {}).'.format(export_task.id))
    time.sleep(30)

print('Export completed.')
Notes:
Correct Asset IDs: Replace 'NEON/D02/OSBS/2016_SDR' and 'NEON/D02/OSBS/2016_RGB' with the correct asset IDs for your NEON datasets. If you're not sure what the correct asset IDs are, you can search for them in the Earth Engine Data Catalog.
Region of Interest (ROI): Ensure the ROI geometry is correctly defined.
Folder: Make sure the folder GEE_Exports exists in your Google Drive, or replace it with the correct folder name where you want to save the export.
Checking Available Datasets
If you're not sure about the available datasets, you can explore the Earth Engine Data Catalog:

Go to the Earth Engine Data Catalog
Search for relevant datasets by keyword (e.g., "NEON")
Note the dataset IDs and use them in your script.
User
HttpError                                 Traceback (most recent call last)
/usr/local/lib/python3.10/dist-packages/ee/data.py in _execute_cloud_call(call, num_retries)
    401   try:
--> 402     return call.execute(num_retries=num_retries)
    403   except googleapiclient.errors.HttpError as e:

6 frames
HttpError: <HttpError 400 when requesting https://earthengine.googleapis.com/v1/projects/ee-seeds/image:export?alt=json returned "ImageCollection.load: ImageCollection asset 'NEON/D02/OSBS/2016_SDR' not found (does not exist or caller does not have access).". Details: "ImageCollection.load: ImageCollection asset 'NEON/D02/OSBS/2016_SDR' not found (does not exist or caller does not have access).">

During handling of the above exception, another exception occurred:

EEException                               Traceback (most recent call last)
/usr/local/lib/python3.10/dist-packages/ee/data.py in _execute_cloud_call(call, num_retries)
    402     return call.execute(num_retries=num_retries)
    403   except googleapiclient.errors.HttpError as e:
--> 404     raise _translate_cloud_exception(e)  # pylint: disable=raise-missing-from
    405
    406

EEException: ImageCollection.load: ImageCollection asset 'NEON/D02/OSBS/2016_SDR' not found (does not exist or caller does not have access).
ChatGPT
The error you're encountering indicates that the specified ImageCollection asset 'NEON/D02/OSBS/2016_SDR' does not exist or you don't have access to it. This means you need to verify the correct asset IDs for the datasets you want to use.

Here's how to check and use the correct datasets:

Check the Earth Engine Data Catalog:
Go to the Earth Engine Data Catalog and search for relevant datasets.

Find the Correct Dataset IDs:
Look for the dataset IDs that match your requirements. For example, you can search for NEON datasets or any other datasets you need.

Verify the Dataset Access:
Ensure you have access to the datasets. Some datasets may require specific access permissions.

Example with Correct Dataset IDs:
Assuming you find the correct dataset IDs for SDR and RGB collections, you can replace the placeholders with these IDs. Here's an example script with hypothetical correct IDs:

Define the Image Collections and the Script:
python
Copy code
# Import the Earth Engine library
import ee

# Authenticate and initialize the Earth Engine library
ee.Authenticate()
ee.Initialize()

# Define the image collections (replace with the actual paths if different)
# These are example dataset IDs, you need to replace them with the actual IDs you find in the Data Catalog
aopSDR = ee.ImageCollection('NEON/D02/OSBS/2016_SDR')  # Example ID, replace with actual
aopRGB = ee.ImageCollection('NEON/D02/OSBS/2016_RGB')  # Example ID, replace with actual

# Import the 2019 and 2021 SOAP data
SOAP_2019_sdr = aopSDR.filterDate('2019-01-01', '2019-12-31').filterMetadata('NEON_SITE', 'equals', 'SOAP').first()
SOAP_2021_sdr = aopSDR.filterDate('2021-01-01', '2021-12-31').filterMetadata('NEON_SITE', 'equals', 'SOAP').first()

SOAP_2019_rgb = aopRGB.filterDate('2019-01-01', '2019-12-31').filterMetadata('NEON_SITE', 'equals', 'SOAP').first()
SOAP_2021_rgb = aopRGB.filterDate('2021-01-01', '2021-12-31').filterMetadata('NEON_SITE', 'equals', 'SOAP').first()

# Create an ROI that includes only the pixels covered by both years + adds -30m buffer
full_mask = ee.Image.constant(1).clip(SOAP_2019_sdr.geometry()) \
                .updateMask(SOAP_2019_sdr.select(['B001']).mask()) \
                .updateMask(SOAP_2021_sdr.select(['B001']).mask()) \
                .reduceToVectors(maxPixels=1e13, scale=100)
ROI = full_mask.geometry().buffer(-30)

# Calculate NBR
pre_nbr = SOAP_2019_sdr.normalizedDifference(['B097', 'B365'])
post_nbr = SOAP_2021_sdr.normalizedDifference(['B097', 'B365'])

# Calculate dNBR
dNBR = pre_nbr.subtract(post_nbr)

# Define export parameters
export_params = {
    'image': dNBR,
    'description': 'dNBR_Export',
    'scale': 30,
    'region': ROI,
    'fileFormat': 'GeoTIFF',
    'maxPixels': 1e13,
    'folder': 'GEE_Exports'  # Folder in your Google Drive
}

# Start the export task
export_task = ee.batch.Export.image.toDrive(**export_params)
export_task.start()

# Check the status of the task
import time
while export_task.active():
    print('Polling for task (id: {}).'.format(export_task.id))
    time.sleep(30)

print('Export completed.')
Additional Notes:
Searching for Datasets:
Use the Earth Engine Data Catalog to search for datasets by keywords like "NEON" or any specific terms relevant to your data needs.

Verifying Dataset IDs:
Once you find the correct dataset in the Data Catalog, note its ID and replace the placeholders in the script.

Ensuring Access:
Make sure you have the necessary permissions to access the datasets. Some datasets may require specific access permissions.

By following these steps, you should be able to find the correct datasets and successfully export your data from Google Earth Engine to Google Drive using Colab.

User
Export.image.toDrive(image, description, folder, fileNamePrefix, dimensions, region, scale, crs, crsTransform, maxPixels, shardSize, fileDimensions, skipEmptyTiles, fileFormat, formatOptions, priority) I want to us this code to do the export to drive. But then input the map that is created with this code ### Code and comments adpated from NEON at https://www.neonscience.org/resources/learning-hub/tutorials/intro-aop-data-google-earth-engine-tutorial-series#toggle-36
'''------------------------- Normalized Difference Burn Ratio ----------------------------
The normalized burn ratio (NBR) is a normalized difference index using the shortwave-infrared (SWIR) and near-infrared (NIR) portions of the electromagnetic spectrum.
dNBR can be used as a metric to map fire extent and burn severity when calculating the difference between pre and post fire conditions.
'''

# Import the 2019 and 2021 SOAP data
SOAP_2019_sdr = aopSDR \
  .filterDate('2019-01-01', '2019-12-31') \
  .filterMetadata('NEON_SITE', 'equals', 'SOAP') \
  .first()

SOAP_2021_sdr = aopSDR \
  .filterDate('2021-01-01', '2021-12-31') \
  .filterMetadata('NEON_SITE', 'equals', 'SOAP') \
  .first()

SOAP_2019_rgb = aopRGB \
  .filterDate('2019-01-01', '2019-12-31') \
  .filterMetadata('NEON_SITE', 'equals', 'SOAP') \
  .first()


SOAP_2021_rgb = aopRGB \
  .filterDate('2021-01-01', '2021-12-31') \
  .filterMetadata('NEON_SITE', 'equals', 'SOAP') \
  .first()


## Create an ROI that talks only the pixels covered by both years + adds -30m buffer
full_mask = ee.Image.constant(1).clip(SOAP_2019_sdr.geometry()) \
                .updateMask(SOAP_2019_sdr.select(['B001']).mask()) \
                .updateMask(SOAP_2021_sdr.select(['B001']).mask()) \
                .reduceToVectors(maxPixels=1e13, scale=100)
ROI = full_mask.geometry().buffer(-30)

# calculate NBR
# B097: B365:
pre_nbr = SOAP_2019_sdr.normalizedDifference(['B097', 'B365'])
post_nbr = SOAP_2021_sdr.normalizedDifference(['B097', 'B365'])

# calculate dNBR
dNBR = pre_nbr \
        .subtract(post_nbr)

# Mapping
vis_params = {'min':-1, 'max':1, 'palette':cm.get_palette('RdYlGn_r', n_class=20)}
m = geemap.Map()


# # add RGB base
m.addLayer(SOAP_2019_rgb, rgbVis, 'SOAP 2019 RGB')
m.addLayer(SOAP_2021_rgb, rgbVis, 'SOAP 2021 RGB')

# # add NBR layers
m.addLayer(pre_nbr, vis_params, '2019 Normalized Burn Ratio')
m.addLayer(post_nbr, vis_params, '2021 Normalized Burn Ratio')

# add dNBR layer
m.addLayer(dNBR, vis_params, 'dNBR')

m.centerObject(ROI)
m
ChatGPT
To export the dNBR image created in your script to Google Drive using the Export.image.toDrive function, you need to set up the export parameters and initiate the export task. Below is the complete code to do this:

Ensure all necessary libraries are imported and authenticated:
python
Copy code
import ee
import geemap
import time

# Authenticate and initialize the Earth Engine library
ee.Authenticate()
ee.Initialize()
Define the main script to process and visualize the data:
python
Copy code
# Define the image collections (replace with the actual paths if different)
aopSDR = ee.ImageCollection('YOUR_AOP_SDR_COLLECTION_ID')  # Replace with actual path
aopRGB = ee.ImageCollection('YOUR_AOP_RGB_COLLECTION_ID')  # Replace with actual path

# Import the 2019 and 2021 SOAP data
SOAP_2019_sdr = aopSDR.filterDate('2019-01-01', '2019-12-31').filterMetadata('NEON_SITE', 'equals', 'SOAP').first()
SOAP_2021_sdr = aopSDR.filterDate('2021-01-01', '2021-12-31').filterMetadata('NEON_SITE', 'equals', 'SOAP').first()

SOAP_2019_rgb = aopRGB.filterDate('2019-01-01', '2019-12-31').filterMetadata('NEON_SITE', 'equals', 'SOAP').first()
SOAP_2021_rgb = aopRGB.filterDate('2021-01-01', '2021-12-31').filterMetadata('NEON_SITE', 'equals', 'SOAP').first()

# Create an ROI that includes only the pixels covered by both years + adds -30m buffer
full_mask = ee.Image.constant(1).clip(SOAP_2019_sdr.geometry()) \
                .updateMask(SOAP_2019_sdr.select(['B001']).mask()) \
                .updateMask(SOAP_2021_sdr.select(['B001']).mask()) \
                .reduceToVectors(maxPixels=1e13, scale=100)
ROI = full_mask.geometry().buffer(-30)

# Calculate NBR
pre_nbr = SOAP_2019_sdr.normalizedDifference(['B097', 'B365'])
post_nbr = SOAP_2021_sdr.normalizedDifference(['B097', 'B365'])

# Calculate dNBR
dNBR = pre_nbr.subtract(post_nbr)

# Mapping
rgbVis = {
    'min': 0.0,
    'max': 255.0,
    'bands': ['B04', 'B03', 'B02'],
}
vis_params = {'min': -1, 'max': 1, 'palette': ['red', 'yellow', 'green']}

m = geemap.Map()

# Add RGB base
m.addLayer(SOAP_2019_rgb, rgbVis, 'SOAP 2019 RGB')
m.addLayer(SOAP_2021_rgb, rgbVis, 'SOAP 2021 RGB')

# Add NBR layers
m.addLayer(pre_nbr, vis_params, '2019 Normalized Burn Ratio')
m.addLayer(post_nbr, vis_params, '2021 Normalized Burn Ratio')

# Add dNBR layer
m.addLayer(dNBR, vis_params, 'dNBR')

m.centerObject(ROI)
m
Set up the export to Google Drive:
python
Copy code
# Define export parameters
export_params = {
    'image': dNBR,
    'description': 'dNBR_Export',
    'folder': 'GEE_Exports',
    'fileNamePrefix': 'dNBR_Export',
    'scale': 30,
    'region': ROI,
    'fileFormat': 'GeoTIFF',
    'maxPixels': 1e13,
}

# Start the export task
export_task = ee.batch.Export.image.toDrive(**export_params)
export_task.start()

# Check the status of the task
while export_task.active():
    print('Polling for task (id: {}).'.format(export_task.id))
    time.sleep(30)

print('Export completed.')
Complete Example:
Combining everything together, here's the full script:

python
Copy code
import ee
import geemap
import time

# Authenticate and initialize the Earth Engine library
ee.Authenticate()
ee.Initialize()

# Define the image collections (replace with the actual paths if different)
aopSDR = ee.ImageCollection('NEON/D02/OSBS/2016_SDR')  # Example ID, replace with actual
aopRGB = ee.ImageCollection('NEON/D02/OSBS/2016_RGB')  # Example ID, replace with actual

# Import the 2019 and 2021 SOAP data
SOAP_2019_sdr = aopSDR.filterDate('2019-01-01', '2019-12-31').filterMetadata('NEON_SITE', 'equals', 'SOAP').first()
SOAP_2021_sdr = aopSDR.filterDate('2021-01-01', '2021-12-31').filterMetadata('NEON_SITE', 'equals', 'SOAP').first()

SOAP_2019_rgb = aopRGB.filterDate('2019-01-01', '2019-12-31').filterMetadata('NEON_SITE', 'equals', 'SOAP').first()
SOAP_2021_rgb = aopRGB.filterDate('2021-01-01', '2021-12-31').filterMetadata('NEON_SITE', 'equals', 'SOAP').first()

# Create an ROI that includes only the pixels covered by both years + adds -30m buffer
full_mask = ee.Image.constant(1).clip(SOAP_2019_sdr.geometry()) \
                .updateMask(SOAP_2019_sdr.select(['B001']).mask()) \
                .updateMask(SOAP_2021_sdr.select(['B001']).mask()) \
                .reduceToVectors(maxPixels=1e13, scale=100)
ROI = full_mask.geometry().buffer(-30)

# Calculate NBR
pre_nbr = SOAP_2019_sdr.normalizedDifference(['B097', 'B365'])
post_nbr = SOAP_2021_sdr.normalizedDifference(['B097', 'B365'])

# Calculate dNBR
dNBR = pre_nbr.subtract(post_nbr)

# Mapping
rgbVis = {
    'min': 0.0,
    'max': 255.0,
    'bands': ['B04', 'B03', 'B02'],
}
vis_params = {'min': -1, 'max': 1, 'palette': ['red', 'yellow', 'green']}

m = geemap.Map()

# Add RGB base
m.addLayer(SOAP_2019_rgb, rgbVis, 'SOAP 2019 RGB')
m.addLayer(SOAP_2021_rgb, rgbVis, 'SOAP 2021 RGB')

# Add NBR layers
m.addLayer(pre_nbr, vis_params, '2019 Normalized Burn Ratio')
m.addLayer(post_nbr, vis_params, '2021 Normalized Burn Ratio')

# Add dNBR layer
m.addLayer(dNBR, vis_params, 'dNBR')

m.centerObject(ROI)
m

# Define export parameters
export_params = {
    'image': dNBR,
    'description': 'dNBR_Export',
    'folder': 'GEE_Exports',
    'fileNamePrefix': 'dNBR_Export',
    'scale': 30,
    'region': ROI,
    'fileFormat': 'GeoTIFF',
    'maxPixels': 1e13,
}

# Start the export task
export_task = ee.batch.Export.image.toDrive(**export_params)
export_task.start()

# Check the status of the task
while export_task.active():
    print('Polling for task (id: {}).'.format(export_task.id))
    time.sleep(30)

print('Export completed.')
Notes:
Replace Dataset IDs: Ensure you replace the placeholder dataset IDs with the actual IDs you are using.
Folder Name: Make sure the folder GEE_Exports exists in your Google Drive, or specify a different folder name that exists.
Check the status of the task: The script will poll the status of the export task and print messages until the export is completed.
User
import ee
import geemap
import time

# Define the image collections (replace with the actual paths if different)
aopSDR = ee.ImageCollection('/content/sample_data')  # Replace with actual path
aopRGB = ee.ImageCollection('/content/sample_data')  # Replace with actual path

# Import the 2019 and 2021 SOAP data
SOAP_2019_sdr = aopSDR.filterDate('2019-01-01', '2019-12-31').filterMetadata('NEON_SITE', 'equals', 'SOAP').first()
SOAP_2021_sdr = aopSDR.filterDate('2021-01-01', '2021-12-31').filterMetadata('NEON_SITE', 'equals', 'SOAP').first()

SOAP_2019_rgb = aopRGB.filterDate('2019-01-01', '2019-12-31').filterMetadata('NEON_SITE', 'equals', 'SOAP').first()
SOAP_2021_rgb = aopRGB.filterDate('2021-01-01', '2021-12-31').filterMetadata('NEON_SITE', 'equals', 'SOAP').first()

# Create an ROI that includes only the pixels covered by both years + adds -30m buffer
full_mask = ee.Image.constant(1).clip(SOAP_2019_sdr.geometry()) \
                .updateMask(SOAP_2019_sdr.select(['B001']).mask()) \
                .updateMask(SOAP_2021_sdr.select(['B001']).mask()) \
                .reduceToVectors(maxPixels=1e13, scale=100)
ROI = full_mask.geometry().buffer(-30)

# Calculate NBR
pre_nbr = SOAP_2019_sdr.normalizedDifference(['B097', 'B365'])
post_nbr = SOAP_2021_sdr.normalizedDifference(['B097', 'B365'])

# Calculate dNBR
dNBR = pre_nbr.subtract(post_nbr)

# Mapping
rgbVis = {
    'min': 0.0,
    'max': 255.0,
    'bands': ['B04', 'B03', 'B02'],
}
vis_params = {'min': -1, 'max': 1, 'palette': ['red', 'yellow', 'green']}

m = geemap.Map()

# Add RGB base
m.addLayer(SOAP_2019_rgb, rgbVis, 'SOAP 2019 RGB')
m.addLayer(SOAP_2021_rgb, rgbVis, 'SOAP 2021 RGB')

# Add NBR layers
m.addLayer(pre_nbr, vis_params, '2019 Normalized Burn Ratio')
m.addLayer(post_nbr, vis_params, '2021 Normalized Burn Ratio')

# Add dNBR layer
m.addLayer(dNBR, vis_params, 'dNBR')

m.centerObject(ROI)
m
---------------------------------------------------------------------------
HttpError                                 Traceback (most recent call last)
/usr/local/lib/python3.10/dist-packages/ee/data.py in _execute_cloud_call(call, num_retries)
    401   try:
--> 402     return call.execute(num_retries=num_retries)
    403   except googleapiclient.errors.HttpError as e:

9 frames
HttpError: <HttpError 400 when requesting https://earthengine.googleapis.com/v1/projects/ee-seeds/maps?fields=name&alt=json returned "ImageCollection.load: ImageCollection asset '/content/sample_data' not found (does not exist or caller does not have access).". Details: "ImageCollection.load: ImageCollection asset '/content/sample_data' not found (does not exist or caller does not have access).">

During handling of the above exception, another exception occurred:

EEException                               Traceback (most recent call last)
/usr/local/lib/python3.10/dist-packages/ee/data.py in _execute_cloud_call(call, num_retries)
    402     return call.execute(num_retries=num_retries)
    403   except googleapiclient.errors.HttpError as e:
--> 404     raise _translate_cloud_exception(e)  # pylint: disable=raise-missing-from
    405
    406

EEException: ImageCollection.load: ImageCollection asset '/content/sample_data' not found (does not exist or caller does not have access).
ChatGPT
The error you're encountering is because you are trying to load an ImageCollection from a local path ('/content/sample_data'), which is not valid in the context of Google Earth Engine. Earth Engine assets must be from the Earth Engine Data Catalog or your personal Earth Engine assets.

To proceed, you need to use the actual Earth Engine dataset IDs. Here's how you can modify the script with a proper Earth Engine dataset:

Step-by-Step Solution
Authenticate and Initialize Earth Engine:
python
Copy code
import ee
import geemap
import time

# Authenticate and initialize the Earth Engine library
ee.Authenticate()
ee.Initialize()
Use Correct Dataset IDs:
Replace /content/sample_data with the actual Earth Engine ImageCollection IDs.
Example Script
python
Copy code
# Define the image collections (replace with the actual paths if different)
aopSDR = ee.ImageCollection('NEON/D02/OSBS/DP1.30006.001')  # Example NEON SDR dataset ID, replace with actual
aopRGB = ee.ImageCollection('NEON/D02/OSBS/DP3.30010.001')  # Example NEON RGB dataset ID, replace with actual

# Import the 2019 and 2021 SOAP data
SOAP_2019_sdr = aopSDR.filterDate('2019-01-01', '2019-12-31').filterMetadata('NEON_SITE', 'equals', 'SOAP').first()
SOAP_2021_sdr = aopSDR.filterDate('2021-01-01', '2021-12-31').filterMetadata('NEON_SITE', 'equals', 'SOAP').first()

SOAP_2019_rgb = aopRGB.filterDate('2019-01-01', '2019-12-31').filterMetadata('NEON_SITE', 'equals', 'SOAP').first()
SOAP_2021_rgb = aopRGB.filterDate('2021-01-01', '2021-12-31').filterMetadata('NEON_SITE', 'equals', 'SOAP').first()

# Create an ROI that includes only the pixels covered by both years + adds -30m buffer
full_mask = ee.Image.constant(1).clip(SOAP_2019_sdr.geometry()) \
                .updateMask(SOAP_2019_sdr.select(['B001']).mask()) \
                .updateMask(SOAP_2021_sdr.select(['B001']).mask()) \
                .reduceToVectors(maxPixels=1e13, scale=100)
ROI = full_mask.geometry().buffer(-30)

# Calculate NBR
pre_nbr = SOAP_2019_sdr.normalizedDifference(['B097', 'B365'])
post_nbr = SOAP_2021_sdr.normalizedDifference(['B097', 'B365'])

# Calculate dNBR
dNBR = pre_nbr.subtract(post_nbr)

# Mapping
rgbVis = {
    'min': 0.0,
    'max': 255.0,
    'bands': ['B04', 'B03', 'B02'],
}
vis_params = {'min': -1, 'max': 1, 'palette': ['red', 'yellow', 'green']}

m = geemap.Map()

# Add RGB base
m.addLayer(SOAP_2019_rgb, rgbVis, 'SOAP 2019 RGB')
m.addLayer(SOAP_2021_rgb, rgbVis, 'SOAP 2021 RGB')

# Add NBR layers
m.addLayer(pre_nbr, vis_params, '2019 Normalized Burn Ratio')
m.addLayer(post_nbr, vis_params, '2021 Normalized Burn Ratio')

# Add dNBR layer
m.addLayer(dNBR, vis_params, 'dNBR')

m.centerObject(ROI)
m

# Define export parameters
export_params = {
    'image': dNBR,
    'description': 'dNBR_Export',
    'folder': 'GEE_Exports',
    'fileNamePrefix': 'dNBR_Export',
    'scale': 30,
    'region': ROI,
    'fileFormat': 'GeoTIFF',
    'maxPixels': 1e13,
}

# Start the export task
export_task = ee.batch.Export.image.toDrive(**export_params)
export_task.start()

# Check the status of the task
while export_task.active():
    print('Polling for task (id: {}).'.format(export_task.id))
    time.sleep(30)

print('Export completed.')
Important Notes:
Replace Dataset IDs: The dataset IDs used in this example ('NEON/D02/OSBS/DP1.30006.001' and 'NEON/D02/OSBS/DP3.30010.001') are placeholders. You need to replace them with the actual dataset IDs you want to use. You can find these IDs in the Earth Engine Data Catalog.
Folder in Google Drive: Ensure that the folder GEE_Exports exists in your Google Drive or replace it with the correct folder name where you want to save the export.
This script will load the correct datasets, create the dNBR image, visualize it on the map, and export it to Google Drive.

User
# Define the image collections (replace with the actual paths if different)
aopSDR = ee.ImageCollection('projects/neon-prod-earthengine/assets/DP3-30006-001')  # Example NEON SDR dataset ID, replace with actual
aopRGB = ee.ImageCollection('projects/neon-prod-earthengine/assets/DP3-30010-001')  # Example NEON RGB dataset ID, replace with actual

# Import the 2019 and 2021 SOAP data
SOAP_2019_sdr = aopSDR.filterDate('2019-01-01', '2019-12-31').filterMetadata('NEON_SITE', 'equals', 'SOAP').first()
SOAP_2021_sdr = aopSDR.filterDate('2021-01-01', '2021-12-31').filterMetadata('NEON_SITE', 'equals', 'SOAP').first()

SOAP_2019_rgb = aopRGB.filterDate('2019-01-01', '2019-12-31').filterMetadata('NEON_SITE', 'equals', 'SOAP').first()
SOAP_2021_rgb = aopRGB.filterDate('2021-01-01', '2021-12-31').filterMetadata('NEON_SITE', 'equals', 'SOAP').first()

# Create an ROI that includes only the pixels covered by both years + adds -30m buffer
full_mask = ee.Image.constant(1).clip(SOAP_2019_sdr.geometry()) \
                .updateMask(SOAP_2019_sdr.select(['B001']).mask()) \
                .updateMask(SOAP_2021_sdr.select(['B001']).mask()) \
                .reduceToVectors(maxPixels=1e13, scale=100)
ROI = full_mask.geometry().buffer(-30)

# Calculate NBR
pre_nbr = SOAP_2019_sdr.normalizedDifference(['B097', 'B365'])
post_nbr = SOAP_2021_sdr.normalizedDifference(['B097', 'B365'])

# Calculate dNBR
dNBR = pre_nbr.subtract(post_nbr)

# Mapping
rgbVis = {
    'min': 0.0,
    'max': 255.0,
    'bands': ['B04', 'B03', 'B02'],
}
vis_params = {'min': -1, 'max': 1, 'palette': ['red', 'yellow', 'green']}

m = geemap.Map()

# Add RGB base
m.addLayer(SOAP_2019_rgb, rgbVis, 'SOAP 2019 RGB')
m.addLayer(SOAP_2021_rgb, rgbVis, 'SOAP 2021 RGB')

# Add NBR layers
m.addLayer(pre_nbr, vis_params, '2019 Normalized Burn Ratio')
m.addLayer(post_nbr, vis_params, '2021 Normalized Burn Ratio')

# Add dNBR layer
m.addLayer(dNBR, vis_params, 'dNBR')

m.centerObject(ROI)
m

# Define export parameters
export_params = {
    'image': dNBR,
    'description': 'dNBR_Export',
    'folder': 'GEE_Exports',
    'fileNamePrefix': 'dNBR_Export',
    'scale': 30,
    'region': ROI,
    'fileFormat': 'GeoTIFF',
    'maxPixels': 1e13,
}

# Start the export task
export_task = ee.batch.Export.image.toDrive(**export_params)
export_task.start()

# Check the status of the task
while export_task.active():
    print('Polling for task (id: {}).'.format(export_task.id))
    time.sleep(30)

print('Export completed.')
---------------------------------------------------------------------------
HttpError                                 Traceback (most recent call last)
/usr/local/lib/python3.10/dist-packages/ee/data.py in _execute_cloud_call(call, num_retries)
    401   try:
--> 402     return call.execute(num_retries=num_retries)
    403   except googleapiclient.errors.HttpError as e:

9 frames
HttpError: <HttpError 400 when requesting https://earthengine.googleapis.com/v1/projects/ee-seeds/maps?fields=name&alt=json returned "Image.visualize: No band named 'B04'. Available band names: [R, G, B].". Details: "Image.visualize: No band named 'B04'. Available band names: [R, G, B].">

During handling of the above exception, another exception occurred:

EEException                               Traceback (most recent call last)
/usr/local/lib/python3.10/dist-packages/ee/data.py in _execute_cloud_call(call, num_retries)
    402     return call.execute(num_retries=num_retries)
    403   except googleapiclient.errors.HttpError as e:
--> 404     raise _translate_cloud_exception(e)  # pylint: disable=raise-missing-from
    405
    406

EEException: Image.visualize: No band named 'B04'. Available band names: [R, G, B].
ChatGPT
The error you're encountering indicates that the RGB image does not have bands named B04, B03, and B02. Instead, it has bands named R, G, and B.

Here's how you can modify your script to use the correct band names:

Update the Visualization Parameters:
Use the correct band names for the RGB images (R, G, and B).

Adjust the Script:

python
Copy code
import ee
import geemap
import time

# Authenticate and initialize the Earth Engine library
ee.Authenticate()
ee.Initialize()

# Define the image collections (replace with the actual paths if different)
aopSDR = ee.ImageCollection('projects/neon-prod-earthengine/assets/DP3-30006-001')  # NEON SDR dataset ID
aopRGB = ee.ImageCollection('projects/neon-prod-earthengine/assets/DP3-30010-001')  # NEON RGB dataset ID

# Import the 2019 and 2021 SOAP data
SOAP_2019_sdr = aopSDR.filterDate('2019-01-01', '2019-12-31').filterMetadata('NEON_SITE', 'equals', 'SOAP').first()
SOAP_2021_sdr = aopSDR.filterDate('2021-01-01', '2021-12-31').filterMetadata('NEON_SITE', 'equals', 'SOAP').first()

SOAP_2019_rgb = aopRGB.filterDate('2019-01-01', '2019-12-31').filterMetadata('NEON_SITE', 'equals', 'SOAP').first()
SOAP_2021_rgb = aopRGB.filterDate('2021-01-01', '2021-12-31').filterMetadata('NEON_SITE', 'equals', 'SOAP').first()

# Create an ROI that includes only the pixels covered by both years + adds -30m buffer
full_mask = ee.Image.constant(1).clip(SOAP_2019_sdr.geometry()) \
                .updateMask(SOAP_2019_sdr.select(['B001']).mask()) \
                .updateMask(SOAP_2021_sdr.select(['B001']).mask()) \
                .reduceToVectors(maxPixels=1e13, scale=100)
ROI = full_mask.geometry().buffer(-30)

# Calculate NBR
pre_nbr = SOAP_2019_sdr.normalizedDifference(['B097', 'B365'])
post_nbr = SOAP_2021_sdr.normalizedDifference(['B097', 'B365'])

# Calculate dNBR
dNBR = pre_nbr.subtract(post_nbr)

# Mapping
rgbVis = {
    'min': 0.0,
    'max': 255.0,
    'bands': ['R', 'G', 'B'],
}
vis_params = {'min': -1, 'max': 1, 'palette': ['red', 'yellow', 'green']}

m = geemap.Map()

# Add RGB base
m.addLayer(SOAP_2019_rgb, rgbVis, 'SOAP 2019 RGB')
m.addLayer(SOAP_2021_rgb, rgbVis, 'SOAP 2021 RGB')

# Add NBR layers
m.addLayer(pre_nbr, vis_params, '2019 Normalized Burn Ratio')
m.addLayer(post_nbr, vis_params, '2021 Normalized Burn Ratio')

# Add dNBR layer
m.addLayer(dNBR, vis_params, 'dNBR')

m.centerObject(ROI)
m

# Define export parameters
export_params = {
    'image': dNBR,
    'description': 'dNBR_Export',
    'folder': 'GEE_Exports',
    'fileNamePrefix': 'dNBR_Export',
    'scale': 30,
    'region': ROI,
    'fileFormat': 'GeoTIFF',
    'maxPixels': 1e13,
}

# Start the export task
export_task = ee.batch.Export.image.toDrive(**export_params)
export_task.start()

# Check the status of the task
while export_task.active():
    print('Polling for task (id: {}).'.format(export_task.id))
    time.sleep(30)

print('Export completed.')
Explanation:
Dataset Initialization:

The aopSDR and aopRGB variables are initialized with the correct NEON dataset IDs.
Visualization Parameters:

The rgbVis dictionary is updated to use the bands R, G, and B for visualization.
Mapping:

The script visualizes the RGB images and the NBR layers correctly.
Export to Google Drive:

The dNBR image is exported to Google Drive with the specified parameters.
This script should now work without errors, provided that the dataset IDs and the band names are correctly specified. Ensure that you have access to the specified datasets and that they contain the required bands.

User
Is there a way to export it as a pdf
ChatGPT
Exporting an image directly as a PDF from Google Earth Engine (GEE) is not supported. However, you can export the image as a GeoTIFF and then convert it to a PDF using Python libraries such as Matplotlib and Rasterio.

Here's how you can do it step-by-step:

Export the image from GEE as a GeoTIFF.
Convert the GeoTIFF to a PDF using Python.
Step 1: Export the image as a GeoTIFF
Here's the script to export the dNBR image from GEE:

python
Copy code
import ee
import geemap
import time

# Authenticate and initialize the Earth Engine library
ee.Authenticate()
ee.Initialize()

# Define the image collections (replace with the actual paths if different)
aopSDR = ee.ImageCollection('projects/neon-prod-earthengine/assets/DP3-30006-001')  # NEON SDR dataset ID
aopRGB = ee.ImageCollection('projects/neon-prod-earthengine/assets/DP3-30010-001')  # NEON RGB dataset ID

# Import the 2019 and 2021 SOAP data
SOAP_2019_sdr = aopSDR.filterDate('2019-01-01', '2019-12-31').filterMetadata('NEON_SITE', 'equals', 'SOAP').first()
SOAP_2021_sdr = aopSDR.filterDate('2021-01-01', '2021-12-31').filterMetadata('NEON_SITE', 'equals', 'SOAP').first()

SOAP_2019_rgb = aopRGB.filterDate('2019-01-01', '2019-12-31').filterMetadata('NEON_SITE', 'equals', 'SOAP').first()
SOAP_2021_rgb = aopRGB.filterDate('2021-01-01', '2021-12-31').filterMetadata('NEON_SITE', 'equals', 'SOAP').first()

# Create an ROI that includes only the pixels covered by both years + adds -30m buffer
full_mask = ee.Image.constant(1).clip(SOAP_2019_sdr.geometry()) \
                .updateMask(SOAP_2019_sdr.select(['B001']).mask()) \
                .updateMask(SOAP_2021_sdr.select(['B001']).mask()) \
                .reduceToVectors(maxPixels=1e13, scale=100)
ROI = full_mask.geometry().buffer(-30)

# Calculate NBR
pre_nbr = SOAP_2019_sdr.normalizedDifference(['B097', 'B365'])
post_nbr = SOAP_2021_sdr.normalizedDifference(['B097', 'B365'])

# Calculate dNBR
dNBR = pre_nbr.subtract(post_nbr)

# Mapping
rgbVis = {
    'min': 0.0,
    'max': 255.0,
    'bands': ['R', 'G', 'B'],
}
vis_params = {'min': -1, 'max': 1, 'palette': ['red', 'yellow', 'green']}

m = geemap.Map()

# Add RGB base
m.addLayer(SOAP_2019_rgb, rgbVis, 'SOAP 2019 RGB')
m.addLayer(SOAP_2021_rgb, rgbVis, 'SOAP 2021 RGB')

# Add NBR layers
m.addLayer(pre_nbr, vis_params, '2019 Normalized Burn Ratio')
m.addLayer(post_nbr, vis_params, '2021 Normalized Burn Ratio')

# Add dNBR layer
m.addLayer(dNBR, vis_params, 'dNBR')

m.centerObject(ROI)
m

# Define export parameters
export_params = {
    'image': dNBR,
    'description': 'dNBR_Export',
    'folder': 'GEE_Exports',
    'fileNamePrefix': 'dNBR_Export',
    'scale': 30,
    'region': ROI,
    'fileFormat': 'GeoTIFF',
    'maxPixels': 1e13,
}

# Start the export task
export_task = ee.batch.Export.image.toDrive(**export_params)
export_task.start()

# Check the status of the task
while export_task.active():
    print('Polling for task (id: {}).'.format(export_task.id))
    time.sleep(30)

print('Export completed.')
Step 2: Convert the GeoTIFF to a PDF using Python
Once the GeoTIFF is exported to Google Drive, you can download it and convert it to a PDF using Python. You need libraries such as rasterio and matplotlib.

First, install the necessary libraries if you haven't already:

bash
Copy code
pip install rasterio matplotlib
Then, use the following script to convert the GeoTIFF to a PDF:

python
Copy code
import rasterio
import matplotlib.pyplot as plt

# Path to the exported GeoTIFF file (download it from Google Drive first)
geotiff_path = 'path/to/downloaded/dNBR_Export.tif'
pdf_path = 'dNBR_Export.pdf'

# Read the GeoTIFF file
with rasterio.open(geotiff_path) as src:
    image = src.read(1)
    fig, ax = plt.subplots(figsize=(10, 10))
    cax = ax.imshow(image, cmap='RdYlGn', vmin=-1, vmax=1)
    ax.set_title('dNBR')
    fig.colorbar(cax, orientation='vertical', label='dNBR')
    plt.savefig(pdf_path, format='pdf')
    plt.close()

print(f'PDF saved as {pdf_path}')
Steps:
Export the GeoTIFF from GEE: Use the script provided to export the dNBR image to Google Drive as a GeoTIFF.
Download the GeoTIFF: Download the GeoTIFF file from Google Drive to your local machine or Colab environment.
Convert to PDF: Run the provided Python script to read the GeoTIFF and save it as a PDF.
This approach leverages the powerful visualization capabilities of matplotlib and rasterio to convert the exported GeoTIFF into a well-formatted PDF.

User
# Define the image collections (replace with the actual paths if different)
aopSDR = ee.ImageCollection('projects/neon-prod-earthengine/assets/DP3-30006-001')  # Example NEON SDR dataset ID, replace with actual
aopRGB = ee.ImageCollection('projects/neon-prod-earthengine/assets/DP3-30010-001')  # Example NEON RGB dataset ID, replace with actual

# Import the 2019 and 2021 SOAP data
SOAP_2019_sdr = aopSDR.filterDate('2019-01-01', '2019-12-31').filterMetadata('NEON_SITE', 'equals', 'SOAP').first()
SOAP_2021_sdr = aopSDR.filterDate('2021-01-01', '2021-12-31').filterMetadata('NEON_SITE', 'equals', 'SOAP').first()

SOAP_2019_rgb = aopRGB.filterDate('2019-01-01', '2019-12-31').filterMetadata('NEON_SITE', 'equals', 'SOAP').first()
SOAP_2021_rgb = aopRGB.filterDate('2021-01-01', '2021-12-31').filterMetadata('NEON_SITE', 'equals', 'SOAP').first()

# Create an ROI that includes only the pixels covered by both years + adds -30m buffer
full_mask = ee.Image.constant(1).clip(SOAP_2019_sdr.geometry()) \
                .updateMask(SOAP_2019_sdr.select(['B001']).mask()) \
                .updateMask(SOAP_2021_sdr.select(['B001']).mask()) \
                .reduceToVectors(maxPixels=1e13, scale=100)
ROI = full_mask.geometry().buffer(-30)

# Calculate NBR
pre_nbr = SOAP_2019_sdr.normalizedDifference(['B097', 'B365'])
post_nbr = SOAP_2021_sdr.normalizedDifference(['B097', 'B365'])

# Calculate dNBR
dNBR = pre_nbr.subtract(post_nbr)

# Mapping
rgbVis = {
    'min': 0.0,
    'max': 255.0,
    'bands': ['R', 'G', 'B'],
}
vis_params = {'min': -1, 'max': 1, 'palette': ['red', 'yellow', 'green']}

m = geemap.Map()

# Add RGB base
m.addLayer(SOAP_2019_rgb, rgbVis, 'SOAP 2019 RGB')
m.addLayer(SOAP_2021_rgb, rgbVis, 'SOAP 2021 RGB')

# Add NBR layers
m.addLayer(pre_nbr, vis_params, '2019 Normalized Burn Ratio')
m.addLayer(post_nbr, vis_params, '2021 Normalized Burn Ratio')

# Add dNBR layer
m.addLayer(dNBR, vis_params, 'dNBR')

m.centerObject(ROI)
m

# Define export parameters
export_params = {
    'image': dNBR,
    'description': 'dNBR_Export',
    'folder': 'GEE_Exports',
    'fileNamePrefix': 'dNBR_Export',
    'scale': 30,
    'region': ROI,
    'fileFormat': 'GeoTIFF',
    'maxPixels': 1e13,
}

# Start the export task
export_task = ee.batch.Export.image.toDrive(**export_params)
export_task.start()

# Check the status of the task
while export_task.active():
    print('Polling for task (id: {}).'.format(export_task.id))
    time.sleep(30)

print('Export completed.')
Polling for task (id: 5UKZIWSIJNPSC3N6EKPNWBQP).
---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<ipython-input-16-2f720e502284> in <cell line: 67>()
     67 while export_task.active():
     68     print('Polling for task (id: {}).'.format(export_task.id))
---> 69     time.sleep(30)
     70
     71 print('Export completed.')

NameError: name 'time' is not defined
ChatGPT
The error you're encountering is because the time module is not imported. You need to import the time module to use the time.sleep function.

Here is the corrected code with the necessary import:

python
Copy code
import ee
import geemap
import time

# Authenticate and initialize the Earth Engine library
ee.Authenticate()
ee.Initialize()

# Define the image collections (replace with the actual paths if different)
aopSDR = ee.ImageCollection('projects/neon-prod-earthengine/assets/DP3-30006-001')  # NEON SDR dataset ID
aopRGB = ee.ImageCollection('projects/neon-prod-earthengine/assets/DP3-30010-001')  # NEON RGB dataset ID

# Import the 2019 and 2021 SOAP data
SOAP_2019_sdr = aopSDR.filterDate('2019-01-01', '2019-12-31').filterMetadata('NEON_SITE', 'equals', 'SOAP').first()
SOAP_2021_sdr = aopSDR.filterDate('2021-01-01', '2021-12-31').filterMetadata('NEON_SITE', 'equals', 'SOAP').first()

SOAP_2019_rgb = aopRGB.filterDate('2019-01-01', '2019-12-31').filterMetadata('NEON_SITE', 'equals', 'SOAP').first()
SOAP_2021_rgb = aopRGB.filterDate('2021-01-01', '2021-12-31').filterMetadata('NEON_SITE', 'equals', 'SOAP').first()

# Create an ROI that includes only the pixels covered by both years + adds -30m buffer
full_mask = ee.Image.constant(1).clip(SOAP_2019_sdr.geometry()) \
                .updateMask(SOAP_2019_sdr.select(['B001']).mask()) \
                .updateMask(SOAP_2021_sdr.select(['B001']).mask()) \
                .reduceToVectors(maxPixels=1e13, scale=100)
ROI = full_mask.geometry().buffer(-30)

# Calculate NBR
pre_nbr = SOAP_2019_sdr.normalizedDifference(['B097', 'B365'])
post_nbr = SOAP_2021_sdr.normalizedDifference(['B097', 'B365'])

# Calculate dNBR
dNBR = pre_nbr.subtract(post_nbr)

# Mapping
rgbVis = {
    'min': 0.0,
    'max': 255.0,
    'bands': ['R', 'G', 'B'],
}
vis_params = {'min': -1, 'max': 1, 'palette': ['red', 'yellow', 'green']}

m = geemap.Map()

# Add RGB base
m.addLayer(SOAP_2019_rgb, rgbVis, 'SOAP 2019 RGB')
m.addLayer(SOAP_2021_rgb, rgbVis, 'SOAP 2021 RGB')

# Add NBR layers
m.addLayer(pre_nbr, vis_params, '2019 Normalized Burn Ratio')
m.addLayer(post_nbr, vis_params, '2021 Normalized Burn Ratio')

# Add dNBR layer
m.addLayer(dNBR, vis_params, 'dNBR')

m.centerObject(ROI)
m

# Define export parameters
export_params = {
    'image': dNBR,
    'description': 'dNBR_Export',
    'folder': 'GEE_Exports',
    'fileNamePrefix': 'dNBR_Export',
    'scale': 30,
    'region': ROI,
    'fileFormat': 'GeoTIFF',
    'maxPixels': 1e13,
}

# Start the export task
export_task = ee.batch.Export.image.toDrive(**export_params)
export_task.start()

# Check the status of the task
while export_task.active():
    print('Polling for task (id: {}).'.format(export_task.id))
    time.sleep(30)

print('Export completed.')
In this script:

Importing the time module: This allows you to use time.sleep(30) to pause execution for 30 seconds between status checks.
Running the export task: The script exports the dNBR image to Google Drive and checks the status of the export task in a loop.
With these changes, the script should now run correctly and handle the export task status polling without encountering the NameError.

User
# Import the 2019 and 2021 SOAP data
SOAP_2019_sdr = aopSDR \
  .filterDate('2019-01-01', '2019-12-31') \
  .filterMetadata('NEON_SITE', 'equals', 'SOAP') \
  .first()

SOAP_2021_sdr = aopSDR \
  .filterDate('2021-01-01', '2021-12-31') \
  .filterMetadata('NEON_SITE', 'equals', 'SOAP') \
  .first()

SOAP_2019_rgb = aopRGB \
  .filterDate('2019-01-01', '2019-12-31') \
  .filterMetadata('NEON_SITE', 'equals', 'SOAP') \
  .first()


SOAP_2021_rgb = aopRGB \
  .filterDate('2021-01-01', '2021-12-31') \
  .filterMetadata('NEON_SITE', 'equals', 'SOAP') \
  .first()


## Create an ROI that talks only the pixels covered by both years + adds -30m buffer
full_mask = ee.Image.constant(1).clip(SOAP_2019_sdr.geometry()) \
                .updateMask(SOAP_2019_sdr.select(['B001']).mask()) \
                .updateMask(SOAP_2021_sdr.select(['B001']).mask()) \
                .reduceToVectors(maxPixels=1e13, scale=100)
ROI = full_mask.geometry().buffer(-30)

# calculate NBR
# B097: B365:
pre_nbr = SOAP_2019_sdr.normalizedDifference(['B097', 'B365'])
post_nbr = SOAP_2021_sdr.normalizedDifference(['B097', 'B365'])

# calculate dNBR
dNBR = pre_nbr \
        .subtract(post_nbr)

# Mapping
vis_params = {'min':-1, 'max':1, 'palette':cm.get_palette('RdYlGn_r', n_class=20)}
m = geemap.Map()


# # add RGB base
m.addLayer(SOAP_2019_rgb, rgbVis, 'SOAP 2019 RGB')
m.addLayer(SOAP_2021_rgb, rgbVis, 'SOAP 2021 RGB')

# # add NBR layers
m.addLayer(pre_nbr, vis_params, '2019 Normalized Burn Ratio')
m.addLayer(post_nbr, vis_params, '2021 Normalized Burn Ratio')

# add dNBR layer
m.addLayer(dNBR, vis_params, 'dNBR')

m.centerObject(ROI)
m # Define export parameters
export_params = {
    'image': dNBR,
    'description': 'dNBR_Export',
    'folder': 'GEE_Exports',
    'fileNamePrefix': 'dNBR_Export',
    'scale': 30,
    'region': ROI,
    'fileFormat': 'GeoTIFF',
    'maxPixels': 1e13,
}

# Start the export task
export_task = ee.batch.Export.image.toDrive(**export_params)
export_task.start()

# Check the status of the task
while export_task.active():
    print('Polling for task (id: {}).'.format(export_task.id))
    time.sleep(30)

print('Export completed.')
Here I was able to export The exported file came out black and white. Is there a way to export it showing all of the layers and colors used in the google earth engine map
ChatGPT
To export an image from Google Earth Engine (GEE) with specific visualization parameters (such as colors and layers), you need to first apply the visualization parameters to the image before exporting it. This can be done using the visualize() method, which creates an RGB image based on the specified visualization parameters.

Here's how you can modify your code to export the dNBR image with the applied visualization parameters:

Apply the Visualization Parameters to the Image:
Export the Visualized Image to Google Drive.
Step-by-Step Solution
Step 1: Apply the Visualization Parameters
Use the visualize() method to create an RGB image based on the specified visualization parameters.

python
Copy code
import ee
import geemap
import time

# Authenticate and initialize the Earth Engine library
ee.Authenticate()
ee.Initialize()

# Define the image collections (replace with the actual paths if different)
aopSDR = ee.ImageCollection('projects/neon-prod-earthengine/assets/DP3-30006-001')  # NEON SDR dataset ID
aopRGB = ee.ImageCollection('projects/neon-prod-earthengine/assets/DP3-30010-001')  # NEON RGB dataset ID

# Import the 2019 and 2021 SOAP data
SOAP_2019_sdr = aopSDR.filterDate('2019-01-01', '2019-12-31').filterMetadata('NEON_SITE', 'equals', 'SOAP').first()
SOAP_2021_sdr = aopSDR.filterDate('2021-01-01', '2021-12-31').filterMetadata('NEON_SITE', 'equals', 'SOAP').first()

SOAP_2019_rgb = aopRGB.filterDate('2019-01-01', '2019-12-31').filterMetadata('NEON_SITE', 'equals', 'SOAP').first()
SOAP_2021_rgb = aopRGB.filterDate('2021-01-01', '2021-12-31').filterMetadata('NEON_SITE', 'equals', 'SOAP').first()

# Create an ROI that includes only the pixels covered by both years + adds -30m buffer
full_mask = ee.Image.constant(1).clip(SOAP_2019_sdr.geometry()) \
                .updateMask(SOAP_2019_sdr.select(['B001']).mask()) \
                .updateMask(SOAP_2021_sdr.select(['B001']).mask()) \
                .reduceToVectors(maxPixels=1e13, scale=100)
ROI = full_mask.geometry().buffer(-30)

# Calculate NBR
pre_nbr = SOAP_2019_sdr.normalizedDifference(['B097', 'B365'])
post_nbr = SOAP_2021_sdr.normalizedDifference(['B097', 'B365'])

# Calculate dNBR
dNBR = pre_nbr.subtract(post_nbr)

# Visualization parameters
vis_params = {'min': -1, 'max': 1, 'palette': ['red', 'yellow', 'green']}

# Apply visualization parameters to create an RGB image
dNBR_visualized = dNBR.visualize(**vis_params)

# Define export parameters
export_params = {
    'image': dNBR_visualized,
    'description': 'dNBR_Export',
    'folder': 'GEE_Exports',
    'fileNamePrefix': 'dNBR_Export',
    'scale': 30,
    'region': ROI,
    'fileFormat': 'GeoTIFF',
    'maxPixels': 1e13,
}

# Start the export task
export_task = ee.batch.Export.image.toDrive(**export_params)
export_task.start()

# Check the status of the task
while export_task.active():
    print('Polling for task (id: {}).'.format(export_task.id))
    time.sleep(30)

print('Export completed.')
Explanation:
Apply Visualization Parameters:

The dNBR.visualize(**vis_params) method is used to create an RGB image based on the specified visualization parameters (vis_params).
Export the Visualized Image:

The export_params dictionary now references the dNBR_visualized image instead of the raw dNBR image.
This ensures that the exported image retains the color mapping and other visualization settings applied in GEE.
This approach ensures that the exported GeoTIFF file will include the colors and visualization parameters specified, making it easier to interpret the results directly from the exported file.
</details>

## Additional Resources

* **End-to-End Google Earth Engine**: If you'd like to continue exploring the Earth Engine processes and applications, <a href="https://courses.spatialthoughts.com/end-to-end-gee.html#automatic-conversion-of-javascript-code-to-python" target="_blank"> SpatialThoughts Course - Ujaval Gandhi </a> has some nice examples you can follow.

